In [1]:
import sys, os
sys.path.append("../../")

from utils.contract_utils import init_contract
from utils.network_utils import configure_network

import brownie
from brownie.network import chain

In [2]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)
brownie.network.connect('mainnet-fork')


Launching 'ganache-cli --port 8545 --gasLimit 12000000 --accounts 10 --hardfork istanbul --mnemonic brownie --fork https://eth-mainnet.alchemyapi.io/v2/AxU0de70ONfvbD-3_pQX0wUlaBK6g3G4 --chainId 1'...


In [3]:
pool_contract_addr = "0x9D0464996170c6B9e75eED71c68B99dDEDf279e8"  # cvxcrv pool addr

pool_contract = init_contract(contract_addr=pool_contract_addr)
pool_info = pool_contract.info()

initial_A = pool_contract.initial_A()
initial_A_time = pool_contract.initial_A_time()
A = pool_contract.A()
A_precise = pool_contract.A_precise()
A_precision = int(A_precise/A)

print(f"pool param details:")
print(f"initial_A: {initial_A}")
print(f"initial_A_time: {initial_A_time}")
print(f"A: {A}")
print(f"A_precise: {A_precise}")
print(f"A_precision: {A_precision}")

  @title StableSwap
  @notice 2 coin pool implementation with no lending
  @author Curve.Fi
  @details Optimized to only support ERC20's with 18
           decimals that return True/revert

pool param details:
initial_A: 5000
initial_A_time: 0
A: 50
A_precise: 5000
A_precision: 100


# Swap rates before ramp

In [4]:
dx = 100000 * 1e18

In [5]:
pool_contract.get_dy(0, 1, dx)

100414056663484120407537

In [6]:
pool_contract.get_dy(1, 0, dx)

99262302151856324745490

# Ramp param:

In [7]:
sender = brownie.accounts.at(init_contract("0xb9fc157394af804a3578134a6585c0dc9cc990d4").admin(), force=True)
sender

<Account '0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a'>

In [8]:
future_A = int(100)
t0 = chain.time()
t1 = int(t0 + 7 * 86400)

print(f"Ramp parameters: _future_A: {future_A}, _future_time: {t1}, msg.sender: {sender}, pool_contract: {pool_contract_addr}")
pool_contract.ramp_A(future_A, t1, {"from": sender})

chain.sleep(t1)
chain.mine()

Ramp parameters: _future_A: 100, _future_time: 1632672359, msg.sender: 0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a, pool_contract: 0x9D0464996170c6B9e75eED71c68B99dDEDf279e8
Transaction sent: 0x843a114384945ad5a461db0d95469ac8a04a2ae8096b75f33504124d051f0b52
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 896
  Vyper_contract.ramp_A confirmed   Block: 13257111   Gas used: 77174 (0.64%)



13257112

# Swap after ramp

In [9]:
pool_contract.get_dy(0, 1, dx)

100134618717255653560903

In [10]:
pool_contract.get_dy(1, 0, dx)

99552583748775495547054

Disconnect

In [11]:
brownie.network.disconnect()

Terminating local RPC client...
